# Practico 1 - Ejercicio 4

## a) 

Implementacion de FIND-S

### Descripcion del algoritmo FIND-S

El algoritmo Find-S busca la hipótesis más específica que se ajusta a todos los ejemplos positivos de un conjunto de datos. La función `find_s` en el contexto proporcionado implementa este algoritmo en Python y opera de la siguiente manera:

1. **Inicialización de `hip`:** Comienza con '∅' para cada atributo en la lista `hip`, representando la hipótesis más específica. Esta hipótesis inicial es tan restrictiva que, en la práctica, no clasificaría ningún ejemplo como positivo porque excluye todos los posibles valores de los atributos.

2. **Procesamiento de ejemplos positivos:** A medida que itera sobre las filas del DataFrame `data`, la función busca filas donde la columna 'c_x' es igual a 1, indicando un ejemplo positivo.

3. **Actualización de `hip`:** Para un ejemplo positivo, cada atributo en `hip` que es '∅' se reemplaza con el valor correspondiente del ejemplo. Si un atributo ya tiene un valor asignado y este no coincide con el valor del ejemplo positivo actual, se asigna '?' a ese atributo en `hip`. El símbolo '?' hace que la hipótesis sea más general, significando que ese atributo puede tomar cualquier valor, ya que ha habido evidencia contradictoria entre los ejemplos positivos.

4. **Decodificación (si es necesaria):** Después de procesar todos los ejemplos, si hay valores en `hip` que no son '?' ni '∅', se utilizan los `label_encoders` para revertir cualquier codificación previa, devolviendo `hip_decoded` con los valores originales para interpretación humana.

5. **Resultado:** La función devuelve dos versiones de la hipótesis: `hip` en su forma codificada y `hip_decoded` en forma legible, donde la hipótesis representa el conjunto de restricciones más específicas que es consistente con todos los ejemplos positivos observados hasta ese momento.

In [ ]:
def find_s(data, label_encoders):
  hip = ['∅'] * (len(data.columns) - 1)

  for index, row in data.iterrows():
    if row['c_x'] == 1:
      for i in range(len(hip)):
        if hip[i] == '∅':
          hip[i] = row.iloc[i]
        elif hip[i] != row.iloc[i]:
          hip[i] = '?'

  hip_decoded = list(hip)

  for index, value in enumerate(hip_decoded):
    if ['?', '∅'].count(value) == 0:
      label_encoder = label_encoders[data.columns[index]]
      hip_decoded[index] = label_encoder.inverse_transform([value])[0]

  return hip, hip_decoded


## Parte b)

Verificacion de la implementacion con el dataset del teorico.
 

### Representacion de los datos

En el teorico vimos una grilla con valores que representan las diferentes caracteristicas que un estudiante debe afrontar para poder aprobar un examen.

La representacion de los datos es la siguiente:

- `Dedicacion`, `Dificultad`, `Horarios`, `Humedad`, `Humor_Docente`, `C_x`

In [ ]:
# Cargamos el ejemplo de la clase con numpy
import pandas as pd
from sklearn.preprocessing import LabelEncoder

data = pd.read_csv('src/ejemplo_teorico.csv', sep=',')
label_encoders = {}

for column in data.columns:
  label_encoders[column] = LabelEncoder()
  data[column] = label_encoders[column].fit_transform(data[column])

data.head()


In [ ]:
hip_s, hip_s_decoded = find_s(data, label_encoders)
print('Hipótesis S: ', hip_s_decoded)


## c)

Implmente un programa que genere instancias aleatorias y luego las clasifique de acuerdo al concepto:  <?, Media, ?, ?, ?>.

Cuantos ejemplos unicos (sin repetidos) tiene que generar en promedio para aprender el concepto? Cuantos ejemplos unicos positivos?

In [237]:
import random

hip_to_be_learned = ['?', 'Media', '?', '?', '?']
new_rows_set = set()
new_data = pd.DataFrame(columns=data.columns)
hip_s = find_s(new_data, label_encoders)[1]

while hip_to_be_learned != hip_s:

  new_row = [
    random.randint(0, 2),
    random.randint(0, 1),
    random.randint(0, 1),
    random.randint(0, 1),
    random.randint(0, 1),
    0
  ]

  if label_encoders['Dificultad'].inverse_transform([new_row[1]])[0] == 'Media':
    new_row[5] = 1

  new_data.loc[len(new_data)] = new_row
  new_rows_set.add(tuple(new_row))
  hip_s = find_s(new_data, label_encoders)[1]

  print('Hipótesis S: ', hip_s)

print('Nuevas filas distintas: ', len(new_rows_set))

positive_saples = 0

for row in new_rows_set:
  if row[5] == 1:
    positive_saples += 1

print('Nuevas filas positivas distintas: ', positive_saples)


Hipótesis S:  ['∅', '∅', '∅', '∅', '∅']
Hipótesis S:  ['Alta', 'Media', 'Matutino', 'Alta', 'Bueno']
Hipótesis S:  ['Alta', 'Media', 'Matutino', 'Alta', 'Bueno']
Hipótesis S:  ['?', 'Media', 'Matutino', 'Alta', 'Bueno']
Hipótesis S:  ['?', 'Media', 'Matutino', 'Alta', 'Bueno']
Hipótesis S:  ['?', 'Media', 'Matutino', '?', 'Bueno']
Hipótesis S:  ['?', 'Media', '?', '?', '?']
Nuevas filas distintas:  7
Nuevas filas positivas distintas:  4
